In [1]:
import pyvista as pv
import numpy as np
from typing import Tuple, List
import skimage
import trimesh

In [2]:
# read in two different meshes
m1 = pv.read('/Users/josephko/research/smooth-mesh-boolean/data/box_1.obj')
m2 = pv.read('/Users/josephko/research/smooth-mesh-boolean/data/sphere_1.obj')

In [3]:
# plot two meshes on top of each other
pl = pv.Plotter()
pl.add_mesh(m1)
pl.add_mesh(m2)
pl.show()

Widget(value='<iframe src="http://localhost:56831/index.html?ui=P_0x14f2b3770_0&reconnect=auto" class="pyvista…

In [4]:
# _computeAABB function
def _computeAABB(meshes:List[pv.core.pointset.PolyData])->Tuple[int]:
    v = meshes[0].points
    for m in meshes[1:]:
        v = np.concatenate([v, m.points],axis=0)
    minx = np.min(v[:,0])
    maxx = np.max(v[:,0])
    miny = np.min(v[:,1])
    maxy = np.max(v[:,1])
    minz = np.min(v[:,2])
    maxz = np.max(v[:,2])
    return (minx,maxx,miny,maxy,minz,maxz)

aabb = _computeAABB([m1, m2])

In [5]:
resolution = 64
pad = 0.0
ox = pad * (aabb[1]-aabb[0])/resolution
oy = pad * (aabb[3]-aabb[2])/resolution
oz = pad * (aabb[5]-aabb[4])/resolution
x = np.linspace(aabb[0]-ox,aabb[1]+ox,resolution)
y = np.linspace(aabb[2]-oy,aabb[3]+oy,resolution)
z = np.linspace(aabb[4]-oz,aabb[5]+oz,resolution)
points = np.zeros((resolution**3,3))
for i in range(resolution):
    for j in range(resolution):
        for k in range(resolution):
            points[i*resolution**2 + j*resolution + k] = np.asarray([x[i],y[j],z[k]])
print(x.shape, y.shape, z.shape)

(64,) (64,) (64,)


In [6]:
points.shape

(262144, 3)

In [7]:
def _makeSDFGrid(m:pv.core.pointset.PolyData,resolution:int,aabb:Tuple[int],pad:float=0.0)->np.ndarray:
    ox = pad * (aabb[1]-aabb[0])/resolution
    oy = pad * (aabb[3]-aabb[2])/resolution
    oz = pad * (aabb[5]-aabb[4])/resolution
    x = np.linspace(aabb[0]-ox,aabb[1]+ox,resolution)
    y = np.linspace(aabb[2]-oy,aabb[3]+oy,resolution)
    z = np.linspace(aabb[4]-oz,aabb[5]+oz,resolution)
    points = np.zeros((resolution**3,3))
    for i in range(resolution):
        for j in range(resolution):
            for k in range(resolution):
                points[i*resolution**2 + j*resolution + k] = np.asarray([x[i],y[j],z[k]])
    points_vtk = pv.PolyData(points)
    # sdf,_,_,_ = pymesh.signed_distance_to_mesh(m,points)
    sdf_vtk = points_vtk.compute_implicit_distance(m)
    sdf_np = np.array(sdf_vtk['implicit_distance'])
    sdf = np.reshape(sdf_np,(resolution,resolution,resolution))
    return sdf,points

In [8]:
# test function: _makeSDFGrid
resolution = 64
pad = 1.0
sdf1,_ = _makeSDFGrid(m1, resolution, aabb, pad=pad)
sdf2,_ = _makeSDFGrid(m2, resolution, aabb, pad=pad)

In [9]:
sdf = np.minimum(sdf1,sdf2)
ox = pad * (aabb[1]-aabb[0])/resolution
oy = pad * (aabb[3]-aabb[2])/resolution
oz = pad * (aabb[5]-aabb[4])/resolution
spacing = (
    (aabb[1]-aabb[0]+2*ox)/resolution,
    (aabb[3]-aabb[2]+2*oy)/resolution,
    (aabb[5]-aabb[4]+2*oz)/resolution
)
verts,faces,_,_ = skimage.measure.marching_cubes(sdf,level=0.,spacing=spacing)

In [10]:
# save verts and faces
np.save('data/verts', verts)
np.save('data/faces', faces)

In [11]:
# Create a mesh with the given vertices and faces (using trimesh)
mesh = trimesh.Trimesh(vertices=verts, faces=faces)
# save as obj
filepath = '/Users/josephko/research/smooth-mesh-boolean/data/trimesh.obj'
with open(filepath, 'w', encoding='utf-8') as f:
    mesh.export(f, file_type='obj')
# Note: errro creating mesh with pyvista polydata
# mesh = pv.PolyData(verts, faces)
# mesh.plot()

CellSizeError: `faces` cell array size is invalid.